In [ ]:
!pip install torchxrayvision

### Importing some useful libraries

In [ ]:
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import pandas as pd
import torchxrayvision as xrv
import torchvision, torchvision.transforms
import random
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image

Establishing connection to the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading the model for extraction of 18 features

In [ ]:
nn_model = xrv.models.DenseNet(weights="all")
nn_model.op_threshs = None 

Object for resize and cropping of images

In [ ]:
transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

 Defining the path of the folder with the images

In [ ]:
## PLACE THE PATH WHERE ARE FOUND THE IMAGES THAT WILL PASS THROUGH THE MODEL
## (EXTRACTION OF THE 18 PATHOLOGIES)

path_folder_images = '/content/drive/MyDrive/Data - Proyecto - Diplomado/test/normal/'

Saving the names of the images

In [ ]:
path_images = []
filenames = []

for name in os.listdir(path_folder_images):
    path_images.append(path_folder_images + name)
    filenames.append(name)

In [ ]:
filenames[:5]

['NORMAL2-IM-0949-0001.jpeg',
 'NORMAL2-IM-0941-0001.jpeg',
 'NORMAL2-IM-0936-0001.jpeg',
 'NORMAL2-IM-0931-0001.jpeg',
 'NORMAL2-IM-0930-0001.jpeg']

Definition of the dataframe with the names of the images

In [ ]:
dict_images = {'filename': filenames}
df_images = pd.DataFrame(dict_images)
df_images

filename
0    NORMAL2-IM-0949-0001.jpeg
1    NORMAL2-IM-0941-0001.jpeg
2    NORMAL2-IM-0936-0001.jpeg
3    NORMAL2-IM-0931-0001.jpeg
4    NORMAL2-IM-0930-0001.jpeg
..                         ...
239  NORMAL2-IM-0887-0001.jpeg
240  NORMAL2-IM-0893-0001.jpeg
241  NORMAL2-IM-0888-0001.jpeg
242  NORMAL2-IM-0898-0001.jpeg
243  NORMAL2-IM-0897-0001.jpeg

[244 rows x 1 columns]

Defining the path to be saved the above dataframe as a .csv file

In [ ]:
## PLACE THE PATH WHERE THE .csv FILE WILL BE SAVED
## THE .csv FILE CONTAINS THE NAMES OF ALL THE IMAGES -------> path_csv = ' '

path_csv = '/content/drive/MyDrive/Data - Proyecto - Diplomado/test/'
df_images.to_csv(path_csv + 'csv_images.csv', index=False)

Showing one of the images to be analysed

In [ ]:
imagen = xrv.datasets.imread(path_images[0])
plt.imshow(imagen)
plt.show()
print()
print(path_images[0])


/content/drive/MyDrive/Data - Proyecto - Diplomado/test/normal/NORMAL2-IM-0949-0001.jpeg


Definition of the Dataloader

In [ ]:
class ChestImagesDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = read_image(img_path)
        name = self.annotations.iloc[index, 0]

        if self.transform:
            image = self.transform(image)
  
        return image, name

Loading the images

In [ ]:
names_file = path_csv + 'csv_images.csv'

dataset = ChestImagesDataset(csv_file=names_file, root_dir=path_folder_images, transform=transform)
dl = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, num_workers=64, pin_memory=True)

Extracting the 18 pathologies

In [ ]:
dl_18features = []

with torch.no_grad():
    for batch_images, batch_names in dl:
        features = nn_model(batch_images)
        dl_18features.append(features.detach().cpu().numpy())
        print(batch_names)
            
dl_18features = np.concatenate(dl_18features)

('NORMAL2-IM-0949-0001.jpeg', 'NORMAL2-IM-0941-0001.jpeg', 'NORMAL2-IM-0936-0001.jpeg', 'NORMAL2-IM-0931-0001.jpeg', 'NORMAL2-IM-0930-0001.jpeg', 'NORMAL2-IM-0926-0001.jpeg', 'NORMAL2-IM-0923-0001.jpeg', 'NORMAL2-IM-0922-0001.jpeg', 'NORMAL2-IM-0917-0001.jpeg', 'NORMAL2-IM-0909-0001.jpeg', 'NORMAL2-IM-0908-0001.jpeg', 'NORMAL2-IM-0900-0001.jpeg', 'NORMAL2-IM-0993-0001.jpeg', 'NORMAL2-IM-0986-0001.jpeg', 'NORMAL2-IM-0981-0001.jpeg', 'NORMAL2-IM-0983-0001.jpeg', 'NORMAL2-IM-0989-0001.jpeg', 'NORMAL2-IM-0987-0001.jpeg', 'NORMAL2-IM-0971-0001-0002.jpeg', 'NORMAL2-IM-0971-0001-0001.jpeg', 'NORMAL2-IM-0966-0001.jpeg', 'NORMAL2-IM-0962-0001.jpeg', 'NORMAL2-IM-0960-0001.jpeg', 'NORMAL2-IM-0956-0001.jpeg', 'NORMAL2-IM-0954-0001.jpeg', 'NORMAL2-IM-0951-0001.jpeg', 'IM-0027-0001.jpeg', 'IM-0028-0001.jpeg', 'IM-0030-0001.jpeg', 'IM-0033-0001-0002.jpeg', 'IM-0023-0001.jpeg', 'IM-0017-0001.jpeg')
('IM-0010-0001.jpeg', 'IM-0011-0001-0001.jpeg', 'IM-0109-0001.jpeg', 'IM-0061-0001.jpeg', 'IM-0210-0001.

In [ ]:
dl_18features.shape

(244, 18)

Ordering the 18 pathologies for all the images analysed

In [ ]:
Atelectasis = []
Consolidation = []
Infiltration = []
Pneumothorax = []
Edema = []
Emphysema = []
Fibrosis = []
Effusion = []
Pneumonia = []
Pleural_Thickening = []
Cardiomegaly = []
Nodule = []
Mass = []
Hernia = []
Lung_Lesion = []
Fracture = []
Lung_Opacity = []
Enlarged_Cardiomediastinum = []

pathologies = [Atelectasis, Consolidation, Infiltration, Pneumothorax, Edema, Emphysema, Fibrosis, Effusion, Pneumonia, Pleural_Thickening, Cardiomegaly, Nodule, Mass, Hernia, Lung_Lesion, Fracture, Lung_Opacity, Enlarged_Cardiomediastinum]

for i in range(dl_18features.shape[0]):
    for j in range(dl_18features.shape[1]):
        pathologies[j].append(dl_18features[i][j])

Defining the name of the .csv file for saving the 18 pathologies

In [ ]:
## PUT A NAME TO THE .csv FILE THAT WILL CONTAIN THE 18 PATHOLOGIES OF THE IMAGES THAT PASSED THROUGH THE MODEL

name_csv = 'csv_TestNormal_pathologies.csv'

Creating a dataframe with the result of the 18 pathologies and saving it to a .csv file

In [ ]:
dict_pathologies = {'filename': filenames,
                    'Atelectasis': pathologies[0],
                    'Consolidation': pathologies[1],
                    'Infiltration': pathologies[2],
                    'Pneumothorax': pathologies[3],
                    'Edema': pathologies[4],
                    'Emphysema': pathologies[5],
                    'Fibrosis': pathologies[6],
                    'Effusion': pathologies[7],
                    'Pneumonia': pathologies[8],
                    'Pleural_Thickening': pathologies[9],
                    'Cardiomegaly': pathologies[10],
                    'Nodule': pathologies[11],
                    'Mass': pathologies[12],
                    'Hernia': pathologies[13],
                    'Lung Lesion': pathologies[14],
                    'Fracture': pathologies[15],
                    'Lung Opacity': pathologies[16],
                    'Enlarged Cardiomediastinum': pathologies[17]}

df_pathologies = pd.DataFrame(dict_pathologies)

## Cambiar el nombre del archivo .csv dependiendo de la carpeta
df_pathologies.to_csv(path_csv + name_csv, index=False)
df_pathologies

filename  ...  Enlarged Cardiomediastinum
0    NORMAL2-IM-0949-0001.jpeg  ...                   -1.571701
1    NORMAL2-IM-0941-0001.jpeg  ...                   -1.629077
2    NORMAL2-IM-0936-0001.jpeg  ...                   -1.606954
3    NORMAL2-IM-0931-0001.jpeg  ...                   -1.615803
4    NORMAL2-IM-0930-0001.jpeg  ...                   -1.477743
..                         ...  ...                         ...
239  NORMAL2-IM-0887-0001.jpeg  ...                   -1.596818
240  NORMAL2-IM-0893-0001.jpeg  ...                   -1.307994
241  NORMAL2-IM-0888-0001.jpeg  ...                   -1.618786
242  NORMAL2-IM-0898-0001.jpeg  ...                   -1.403285
243  NORMAL2-IM-0897-0001.jpeg  ...                   -1.667219

[244 rows x 19 columns]